In [1]:
import re
import pathlib
import pprint
from typing import Tuple
import io
from io import StringIO
pp = pprint.PrettyPrinter(indent=2, compact=False, width=80)

import numpy as np
import math
import operator
from dataclasses import dataclass

import dateutil.parser
from typing import Union, Tuple, IO, Iterator, List

In [2]:
import extra_modules.typecheck_magic    # %%typecheck

%typecheck_clear

# General principles

## Related functionality should be grouped together

- Namespaced by module: `math.sqrt`, `requests.get`
- Namespaced by class: `'string\n'.strip()`, `datetime.fromtimestamp`

### Interfaces should be discoverable

- Documentation
- Tab completion
- `dir(my_module)`

# General principles

## Think about function signatures

If it isn't obvious from the function signature what your interface does, it's not a great interface

### relativedelta signature:

```python
def relativedelta(dt1: datetime=None, dt2:datetime=None,
                  years: int=0, months: int=0, ...,
                  year: int=0, month: int=0, ...) -> relativedelta:
    ...
```

### Intended use:

```python
relativedelta(dt1: datetime=None, dt2: datetime=None) -> relativedelta
```

**or**

```python
relativedelta(years: int=0, months: int=0, ...,
              year: int=0, month: int=0, ...) -> relativedelta
```

### Return type determined by function logic

```python
def send_request(req: RequestType,
                 return_context: bool) -> Union[ResponseType, Tuple[ResponseType, Context]]:
    ...
```


# General principles

## Provide explicit versions of "magical" interfaces

In [3]:
from dateutil.parser import parse, isoparse
from datetime import datetime

In [4]:
for dt in [parse('March 19, 1951'),
           parse('2019-01-04T12:30Z'),
           parse('2018/03/01')]:
    print(dt)

1951-03-19 00:00:00
2019-01-04 12:30:00+00:00
2018-03-01 00:00:00


In [5]:
isoparse('2019-01-04T12:30Z')

datetime.datetime(2019, 1, 4, 12, 30, tzinfo=tzutc())

In [6]:
try:
    isoparse('2018/03/01')
except Exception as e:
    print(e)

invalid literal for int() with base 10: b'/03'


## Example Class: Coordinate

In [7]:
class Coordinate:
    """Representation of coordinates on a globe in latitude and longitude"""
    def __init__(self, lat, long):
        self.lat = float(lat)
        self.long = float(long)
    
    def __repr__(self):
        return (f'{self.__class__.__name__}(' +
                ','.join(str(v) for v in (self.lat, self.long)) + ')')
    
    def __str__(self):
        latstr = '%f%s' % (abs(self.lat), 'N' if self.lat >= 0.0 else 'S')
        longstr = '%f%s' % (abs(self.long), 'W' if self.long >= 0.0 else 'E' )
        
        return f'{latstr},{longstr}'

_BaseCoordinate = Coordinate

In [8]:
cities = {
    'Tokyo': Coordinate(35.6895, -139.6917),
    'New York': Coordinate(40.7128, 74.0060),
    'Brisbane': Coordinate(-27.4698, -153.0251),
    'Santiago': Coordinate(-33.4489, 70.6693)
}

city_strs = {city: str(coords) for city, coords in cities.items()}
for city, coords in cities.items():
    print(f'{city}: {coords}')

Tokyo: 35.689500N,139.691700E
New York: 40.712800N,74.006000W
Brisbane: 27.469800S,153.025100E
Santiago: 33.448900S,70.669300W


## Multiple ways to construct a Coordinate

In [9]:
class Coordinate(_BaseCoordinate):
    def __init__(self, coord_str_or_lat, long=None):
        if isinstance(coord_str_or_lat, str) and long is None:
            coord_str_or_lat, long = self._parse_coord_str(coord_str_or_lat)
        
        self.lat = float(coord_str_or_lat)
        self.long = float(long)

    @staticmethod
    def _parse_coord_str(coord_str):
        m = re.match('(?P<lat>\d+\.?\d*)(?P<latdir>N|S)' + ',' +
                     '(?P<long>\d+\.?\d*)(?P<longdir>W|E)', coord_str.strip())
        if m is None:
            raise ValueError(f'Invalid coordinates: {coord_str}')
        
        lat = float(m.group('lat')) * (-1 if m.group('latdir') == 'S' else 1)
        long = float(m.group('long')) * (-1 if m.group('longdir') == 'E' else 1)
        
        return lat, long

In [10]:
Coordinate(-27.4698, -153.0251)

Coordinate(-27.4698,-153.0251)

In [11]:
Coordinate('35.689500N,139.691700E')

Coordinate(35.6895,-139.6917)

### Desired API:
- `Coordinate(coord_str: str) -> Coordinate`
- `Coordinate(lat: float, long: float) -> Coordinate`

### Actual API:
- `Coordinate(coord_or_str: Union[float, str], long: float=None) -> Coordinate`

### Errors from naive use

In [12]:
try:
    # long is a valid argument, why is it complaining about converting the coords to floats?!
    Coordinate('35.689500N', -139.00)
except Exception as e:
    print(repr(e))

ValueError("could not convert string to float: '35.689500N'")


In [13]:
try:
    # Why does long have a default value if it's a required argument?
    Coordinate(-18.45)
except Exception as e:
    print(repr(e))

TypeError("float() argument must be a string or a number, not 'NoneType'")


## Using an alternate constructor

In [14]:
class Coordinate(_BaseCoordinate):
    def __init__(self, lat: float, long: float):
        self.lat = float(lat)
        self.long = float(long)
    
    @classmethod
    def from_str(cls, coord_str: str) -> Coordinate:
        m = re.match('(?P<lat>\d+\.?\d*)(?P<latdir>N|S)' + ',' +
                     '(?P<long>\d+\.?\d*)(?P<longdir>W|E)', coord_str.strip())
        if m is None:
            raise ValueError(f'Invalid coordinates: {coord_str}')
        
        lat = float(m.group('lat')) * (-1 if m.group('latdir') == 'S' else 1)
        long = float(m.group('long')) * (-1 if m.group('longdir') == 'E' else 1)
        
        return cls(lat, long)

In [15]:
Coordinate(40.7128, 74.006)

Coordinate(40.7128,74.006)

In [16]:
Coordinate.from_str('40.712800N,74.006000W')

Coordinate(40.7128,74.006)

### API
- `Coordinate(lat: float, long: float) -> Coordinate`
- `Coordinate.from_str(coord_str: str) -> Coordinate`

### Error messages

In [17]:
try:
    Coordinate('35.689500N,139.691700E', -139.00)
except ValueError as e:
    print(repr(e))

ValueError("could not convert string to float: '35.689500N,139.691700E'")


In [18]:
try:
    Coordinate(-18.75)
except TypeError as e:
    print(repr(e))

TypeError("__init__() missing 1 required positional argument: 'long'")


In [19]:
try:
    Coordinate.from_str(-18.75, -139.00)
except TypeError as e:
    print(repr(e))

TypeError('from_str() takes 2 positional arguments but 3 were given')


In [20]:
def round_float_errs(val: float) -> float:
    return round(val*1e8) / 1e8

## Inheriting class behavior

In [21]:
@dataclass
class Point:
    x: float
    y: float
    
    @classmethod
    def from_polar(cls, *args, **kwargs):
        constructor_args = cls.polar_to_cartesian(*args, **kwargs)
        return cls(*constructor_args)
    
    @staticmethod
    def polar_to_cartesian(r, theta):
        x = r * math.cos(theta)
        y = r * math.sin(theta)
        
        # Tweak for floating point errors
        x, y = map(round_float_errs, (x, y))
        return x, y

In [22]:
Point(1.0, 1.0)

Point(x=1.0, y=1.0)

In [23]:
Point.from_polar(math.sqrt(2), math.pi/4)

Point(x=1.0, y=1.0)

Refactoring this into a `classmethod` for the alternate constructor gives the same thing for the `Point` class, but `Point` is no longer hard-coded into the alternate constructor

## Customizing behavior by overriding subclass methods

In [98]:
@dataclass
class NamedPoint(Point):
    x: float
    y: float
    name: str = None

In [99]:
NamedPoint(0, 0, name="origin")

NamedPoint(x=0, y=0, name='origin')

In [100]:
NamedPoint.from_polar(0, 0)

NamedPoint(x=0.0, y=0.0, name=None)

We can now re-use the `from_polar` to get an anonymous `NamedPoint`, but what if we actually want to give our thing a name? For that we can use the `super()` builtin to call the base class constructor first.

In [102]:
@dataclass
class NamedPoint(Point):
    x: float
    y: float
    name: str = None
    
    @classmethod
    def from_polar(cls, *args, name=None, **kwargs):
        # Initialize the base class
        rv = super().from_polar(*args, **kwargs)
        
        # Customize the subclass functions
        if name is not None:
            rv.name = name
        return rv

In [103]:
NamedPoint.from_polar(0, 0, name='origin')

NamedPoint(x=0.0, y=0.0, name='origin')

## Customizing behavior by overriding subclass methods

In [104]:
@dataclass
class Point3D(Point):
    x: float
    y: float
    z: float
        
    @staticmethod
    def polar_to_cartesian(r, theta, phi):
        x = r * math.sin(theta) * math.cos(phi)
        y = r * math.sin(theta) * math.sin(phi)
        z = r * math.cos(theta)
        
        x, y, z = map(round_float_errs, (x, y, z))
        return x, y, z

In [105]:
Point3D(1, 1, 1)

Point3D(x=1, y=1, z=1)

In [106]:
Point3D.from_polar(math.sqrt(3), math.acos(1/math.sqrt(3)), math.atan(1))

Point3D(x=1.0, y=1.0, z=1.0)

# Namespacing functions

In [32]:
import abc

class CartesianBase(metaclass=abc.ABCMeta):
    def __init__(self):
        raise RuntimeError(f"{self.__class__} is a namespace and should not be constructed")
    
    @abc.abstractstaticmethod
    def translate(point, offset, *args, **kwargs):
        """Translate a point `point` by offset `offset`"""
    
    @abc.abstractstaticmethod
    def rotate(point, angle, *args, **kwargs):
        """Rotate a point by angle `angle`"""
    
    @classmethod
    def rotate_in_coords(cls, point, angle, origin=None, *args, **kwargs):
        """Rotate in a new coordinate system defined by translation of the origin"""
        if origin is None:
            return cls.rotate(point, angle, *args)
        
        neg_origin = tuple(map(operator.neg, origin))

        args = tuple(cls.translate(x, neg_origin) for x in args)
        point = cls.translate(point, neg_origin)
        
        rotated = cls.rotate(point, angle, *args)

        return cls.translate(rotated, origin)

In [33]:
def _rotate_3d(point: Tuple[float, float, float],
               angle: float,
               axis: Tuple[float, float, float]) -> Tuple[float, float, float]:
        # lol, too complicated, imma use numpy
        point, u = (np.resize(v, (3, 1)) for v in (point, axis))
        u = u / np.linalg.norm(axis)  # Unit vector
        
        # Construct rotation matrix
        u_x = np.array([[0, -u[2], u[1]],
                        [u[2], 0, -u[0]],
                        [-u[1], u[0], 0]])
        uu = u @ u.T
        R = np.cos(angle) * np.identity(3) + np.sin(angle) * u_x + (1 - np.cos(angle)) * uu
        
        # Apply rotation
        point = R @ point      # Rotate about the axis
        
        return tuple(round_float_errs(c) for c in map(float, point))

def _rotate_2d(point, angle):
        x, y = point
        
        x_out = x * math.cos(angle) - y * math.sin(angle)
        y_out = x * math.sin(angle) + y * math.cos(angle)
        
        # Adjust the 
        x_out, y_out = map(round_float_errs, (x_out, y_out))

        return (x_out, y_out)

In [34]:
class Cartesian2D(CartesianBase):
    @staticmethod
    def translate(point: Tuple[float, float],
                  offset: Tuple[float, float]) -> Tuple[float, float]:
        return (point[0] + offset[0], point[1] + offset[1])
    
    @staticmethod
    def rotate(point: Tuple[float, float], angle: float) -> Tuple[float, float]:
        """Rotate a point by angle `angle` about the origin"""
        return _rotate_2d(point, angle)


In [35]:
class Cartesian3D(CartesianBase):
    @staticmethod
    def translate(point: Tuple[float, float, float],
                  offset: Tuple[float, float, float]) -> Tuple[float, float, float]:
        return tuple(p + o for (p, o) in zip(point, offset))
    
    @staticmethod
    def rotate(point: Tuple[float, float, float],
               angle: float,
               axis: Tuple[float, float, float]=(0.0, 0.0, 1.0)) -> Tuple[float, float, float]:
        """Rotate a point in 3 dimensions by an angle about a given axis"""
        # See the source for the distractingly-long implementation of _rotate_3d
        return _rotate_3d(point, angle, axis)
        


In [36]:
Cartesian2D.rotate((1, 0), math.pi/2)

(0.0, 1.0)

In [37]:
Cartesian3D.rotate((1, 0, 0), math.pi / 2)

(0.0, 1.0, 0.0)

In [38]:
Cartesian2D.rotate_in_coords((1, 0), math.pi / 2, origin=(0, 0.5))

(0.5, 1.5)

In [39]:
Cartesian3D.rotate_in_coords((1, 0, 0), math.pi / 2, origin=(-1, 0, 0))

(-1.0, 2.0, 0.0)

# `classmethod` vs `staticmethod`

### Always use `classmethod` for:

- Alternate constructors
- Functions that reference other class or static methods on the class
- If you need a reference to class properties or attributes
<br/>
<br/>
### Can use `staticmethod` if:

- You are simply namespacing a function under a class
- Any behavior changes in subclasses will be solved with a new implementation


## Can I always just use `classmethod`?
- No significant additional functionality
- Useful for restricting the ways an API can be used

# Functions

What about *function* APIs like this?

In [40]:
# TODO: Add *real life* examples of these things

In [41]:
def print_text(path_or_buffer: Union[str, IO[str]], *args, **kwargs):
    if isinstance(path_or_buffer, str):
        with open(path_or_buffer, 'r') as f:
            print_text(f, *args, **kwargs)
    else:
        print(path_or_buffer.read())

Or like this?

```python
# dateutil.parser.parse's API
def parse(timestr: str, **kwargs) -> Union[datetime.datetime,
                                           Tuple[datetime.datetime, Tuple[str, ...]]]:
    ...
```

In [107]:
dateutil.parser.parse('John Ford was born May 13, 1951 in Remlap, Alabama', fuzzy=True)

datetime.datetime(1951, 5, 13, 0, 0)

In [108]:
dateutil.parser.parse('John Ford was born May 13, 1951 in Remlap, Alabama', fuzzy_with_tokens=True)

(datetime.datetime(1951, 5, 13, 0, 0),
 ('John Ford was born ', ' ', ' ', 'in Remlap, Alabama'))

These types of APIs generally emerge when the task you're trying to achieve has two or more common use cases - like reading from a file path *or* a file-like object, or if a relatively niche workflow needs some information that is discarded as part of the computation process - like making a request and discarding its context; if 1% of people need the context, you don't want to return it 100% of the time and make 99% of people discard it.

In [42]:
def somefunc(x: float, return_intermediate: bool=False) -> Union[float, Tuple[float, float]]:
    intermediate = math.sqrt(x) + 3
    
    rv = round(intermediate ** 2, 2)
    
    if return_intermediate:
        return rv, intermediate
    
    return rv

print(somefunc(3.4))
print(somefunc(3.4, return_intermediate=True))

23.46
(23.46, 4.843908891458577)


## Type checking

In [45]:
%typecheck_clear

In [46]:
%%typecheck run
import math

def somefunc_result(x: float) -> float:
    return somefunc_with_intermediate(x)[0]

def somefunc_with_intermediate(x: float) -> Tuple[float, float]:
    intermediate = math.sqrt(x) + 3
    rv = round(intermediate ** 2, 2)
    
    return rv, intermediate

def somefunc(x: float, return_intermediate: bool=False) -> Union[float, Tuple[float, float]]:
    if return_intermediate:
        return somefunc_with_intermediate(x)
    else:
        return somefunc_result(x)
    
def less_than_5(x: float) -> bool:
    return x < 5.0

In addition to just being kinda complicated, it's also easy for these things to cause problems with type checkers, which may either emit erroneous warnings or miss errors, depending on how you write the static types.

In [47]:
%%typecheck report
less_than_5(somefunc_result(3.5))    # No problem

No type issues


In [48]:
%%typecheck report
less_than_5(somefunc_with_intermediate(2.4))       # Error!
# Argument 1 to "is_foo" has incompatible type "Tuple[str, str]"; expected "str"

less_than_5(somefunc_with_intermediate(2.4)[0]);   # No problem

<string>:22: error: Argument 1 to "less_than_5" has incompatible type "Tuple[float, float]"; expected "float"



In [49]:
%%typecheck run
less_than_5(somefunc(2.4));           # Error!
# Argument 1 to "is_foo" has incompatible type "Union[float, Tuple[float, float]]"; expected "float"

<string>:22: error: Argument 1 to "less_than_5" has incompatible type "Union[float, Tuple[float, float]]"; expected "float"



False

In [50]:
%typecheck_clear

# `variants`

## Desired API
- `print_text(txt: str)` - Print text passed to this function

- `print_text.from_stream(sobj: IO[str])` - Print text from a stream

- `print_text.from_path(path_components: str)` - Open a file on disk and print its contents

In [51]:
import variants

@variants.primary
def print_text(txt: str):
    """Prints any text passed to this function"""
    print(txt)

In [52]:
@print_text.variant('from_stream')
def print_text(sobj: IO[str]):
    """Read text from a stream and print it"""
    print_text(sobj.read())

In [53]:
import pathlib
@print_text.variant('from_path')
def print_text(path_components: Union[str, pathlib.Path]):
    """Open the file specified by `path_components` and print the contents"""
    fpath = pathlib.Path(path_components)
    with open(fpath, 'r') as f:
        print_text.from_stream(f)

## Example use

In [54]:
print_text('Hello, world')

Hello, world


In [55]:
print_text.from_stream(StringIO('Hello, world! This is from a stream!'))

Hello, world! This is from a stream!


In [56]:
print_text.from_path('extras/hello_world.txt')

Hello, world! This is from a file!



## Explicit dispatch

In [57]:
import requests

In [58]:
@print_text.variant('from_url')
def print_text(url):
    r = requests.get(url)
    print_text(r.text)

In [59]:
# Ssssh! (Don't want a *real* runtime dependency on hitting the internet)
@print_text.variant('from_url')
def print_text(url):
    try:
        r = requests.get(url)
    except Exception:
        print_text("Hello, world! (from url)")
    print(r.text)

In [60]:
print_text('Hello, world!')
print_text.from_path('extras/hello_world.txt')
print_text.from_url('https://ganssle.io/files/hello_world.txt')

Hello, world!
Hello, world! This is from a file!

Hello, world! (from url)


In [61]:
_print_text = print_text   # This gets redefined, but I want to keep it around

## Implicit dispatch: `singledispatch`

In [62]:
import functools

In [63]:
print_text = _print_text

In [64]:
@functools.singledispatch
def print_text_sd(txt: str):
    print(txt)

In [65]:
from pathlib import Path

@print_text_sd.register(Path)
def _(path: Path):
    print_text.from_path(path)

In [66]:
@dataclass
class Url:
    url: str

@print_text_sd.register(Url)
def _(url: Url):
    print_text.from_url(url.url)



In [67]:
print_text_sd('Hello, world!')
print_text_sd(Path('extras/hello_world.txt'))
print_text_sd(Url('https://ganssle.io/files/hello_world.txt'))

Hello, world!
Hello, world! This is from a file!

Hello, world! (from url)


### Why not both?

In [68]:
@variants.primary
@functools.singledispatch
def print_text(txt: str):
    print(txt)
    
@print_text.variant('from_path')           # Register the URL variant explicitly
@print_text.register(Path)                 # And with singledispatch!
def print_text(pth: Union[Path, str]):
    with open(pth, 'rt') as f:
        print_text(f.read())

In [69]:
print_text("Hello, world!")

Hello, world!


In [70]:
print_text(Path("extras/hello_world.txt"))
print_text.from_path("extras/hello_world.txt")

Hello, world! This is from a file!

Hello, world! This is from a file!



In [71]:
try:
    print_text.from_path("Hello, world")
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'Hello, world'


## Variation in return type

In [72]:
dtstr = 'It was 3AM on Sept 21, 1986'

In [73]:
dateutil.parser.parse(dtstr, fuzzy=True)

datetime.datetime(1986, 9, 21, 3, 0)

In [74]:
dateutil.parser.parse(dtstr, fuzzy_with_tokens=True)

(datetime.datetime(1986, 9, 21, 3, 0), ('It was ', ' on ', ' ', ' '))

In [110]:
@variants.primary
def fuzzy_parse(dtstr: str, *args, **kwargs):
    kwargs['fuzzy'] = True
    return dateutil.parser.parse(dtstr, *args, **kwargs)

@fuzzy_parse.variant('with_tokens')
def fuzzy_parse(dtstr: str, *args, **kwargs) -> Tuple[datetime, Tuple[str, ...]]:
    kwargs['fuzzy_with_tokens'] = True
    return dateutil.parser.parse(dtstr, *args, **kwargs)

In [111]:
fuzzy_parse(dtstr)

datetime.datetime(1986, 9, 21, 3, 0)

In [112]:
fuzzy_parse.with_tokens(dtstr)

(datetime.datetime(1986, 9, 21, 3, 0), ('It was ', ' on ', ' ', ' '))

## Variation in caching behavior

In [78]:
from functools import lru_cache

@variants.primary
@lru_cache()
def get_config():
    return get_config.nocache()

@get_config.variant('nocache')
def get_config():
    print("Retrieving configuration!")
    return {
        'a1': 'value',
        'b': 12348
    }


In [79]:
a = get_config()

Retrieving configuration!


In [80]:
b = get_config()

In [81]:
c = get_config.nocache()

Retrieving configuration!


## Variation in async behavior

In [82]:
import asyncio
import time

@variants.primary
async def myfunc(n):
    for i in range(n):
        yield i
        await asyncio.sleep(0.5)
        
@myfunc.variant('sync')
def myfunc(n):
    for i in range(n):
        yield i
        time.sleep(0.5)

In [83]:
myfunc(4)

<async_generator object myfunc at 0x7fc164049b70>

In [84]:
myfunc.sync(4)

<generator object myfunc at 0x7fc14a7036d8>

### Interfaces for breaking backwards compatibility

In [85]:
# Python 2.6
class PythonDict(dict):
    def items(self):
        return list(super().items())

In [86]:
d = PythonDict({'one': 1, 'two': 2})
print(d.items())

[('one', 1), ('two', 2)]


In [87]:
# Python 2.7
class PythonDict(dict):
    @variants.primary
    def items(self):
        return self.items.eager()
    
    @items.variant('lazy')
    def items(self):
        return super().items()
    
    @items.variant('eager')
    def items(self):
        return list(self.items.lazy())

In [88]:
d = PythonDict({'one': 1, 'two': 2})

print(d.items())

[('one', 1), ('two', 2)]


In [89]:
print(d.items.lazy())
print(d.items.eager())

dict_items([('one', 1), ('two', 2)])
[('one', 1), ('two', 2)]


In [90]:
# Python 3.0
class PythonDict(dict):
    @variants.primary
    def items(self):
        return self.items.lazy()
    
    @items.variant('lazy')
    def items(self):
        return super().items()
    
    @items.variant('eager')
    def items(self):
        return list(super().items())

In [91]:
d = PythonDict({'one': 1, 'two': 2})
print(d.items())

dict_items([('one', 1), ('two', 2)])


In [92]:
print(d.items.lazy())
print(d.items.eager())

dict_items([('one', 1), ('two', 2)])
[('one', 1), ('two', 2)]


In [93]:
import sys
def warn(msg):
    # Prevent leaking the path of the notebook for cosmetic reasons
    print(msg, file=sys.stderr)

In [94]:
# Python 3.8
class PythonDict(dict):
    @variants.primary
    def items(self):
        return self.items.lazy()
    
    @items.variant('lazy')
    def items(self):
        warn('Explicit use of lazy and eager variants is deprecated!\n' +
             'Use `PythonDict.items()` instead')
        return super().items()
    
    @items.variant('eager')
    def items(self):
        warn('Explicit use of lazy and eager variants is deprecated!\n' +
             'Use `list(PythonDict.items())` instead')
        return list(super().items())

In [95]:
d = PythonDict({'one': 1, 'two': 2})
print(d.items.lazy())

dict_items([('one', 1), ('two', 2)])


Explicit use of lazy and eager variants is deprecated!
Use `PythonDict.items()` instead


In [96]:
print(d.items.eager())

[('one', 1), ('two', 2)]


Explicit use of lazy and eager variants is deprecated!
Use `list(PythonDict.items())` instead


# Syntactic marking of relatedness

## Compare to naming convention (e.g. underscores)

In [113]:
def coordinate_from_string(coord_str: str) -> Coordinate:
    return Coordinate.from_str(coord_str)

In [114]:
def print_text_from_path(fpath: Union[str, pathlib.Path]):
    print_text.from_path(fpath)
    
print_text_from_path('extras/hello_world.txt')

Hello, world! This is from a file!



- Variants are namespaced *under* the primary variant:
    - Tab completion helps discovery of variant functions
    - "Top level" API is kept as clean as possible

## Clean top-level APIs: Documentation

Using the `sphinx_autodoc_variants` sphinx extension:

```rst
.. automodule:: text_print
    :members:
```

![text_print module documentation](images/sphinx_docs_image.png)

## Clean top-level APIs: Completion

### Flat namespace:

![autocomplete - no variants](images/completion_messy.png)

### Function variants:

![autocomplete - no variants](images/completion_clean.png)

![autocomplete - no variants](images/completion_print_text.png)

# `Talk.end()`

# `Talk.end.with_plugs()`

- Find `variants`:
  - PyPI: `variants`: https://pypi.org/project/variants/
  - Github: https://github.com/python-variants/variants

## Upcoming Events

### PyData NYC 2018 - October 17-19
### "Contributing to Open Source: A Guide" - Thursday 10:50AM

### Packaging Sprint at Bloomberg (NYC/London) - October 27/28
- Packaging Sprints: https://wiki.python.org/psf/PackagingSprints